# Análise comparativa de modelos

----
Neste notebook iremos fazer a limpeza e transformação de dados do dataset [Titanic - Machine Learning From Disaster](https://www.kaggle.com/c/titanic/overview), além de realizar uma análise comparativa de modelos.

----

In [1]:
from IPython.display import display, Markdown

import pandas as pd
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn import model_selection
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

## 1. Obtenção de dados

Vamos obter novamente os arquivos brutos de dados e o dicionário criado a partir desses dados antes de iniciar o pré-processamento.

In [2]:
df = pd.read_csv("../data/raw/data.csv")
dicionario = pd.read_csv("../data/external/dictionary.csv")
display(df)
dicionario

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


,variavel,descricao,tipo,subtipo
0,survived,Indica se a pessoa sobreviveu ou não,quantitativa,discreta
1,pclass,Classe do passageiro no navio,qualitativa,ordinal
2,sex,Sexo do passageiro,qualitativa,nominal
3,age,Idade do passageiro,quantitativa,contínua
4,sibsp,Quantidade de irmãos ou cônjuges a bordo do navio,quantitativa,discreta
5,parch,Quantidade de pais ou filhos a bordo do navio,quantitativa,discreta
6,fare,Valor da tarifa do passageiro,quantitativa,contínua
7,embarked,Porto de embarcação do passageiro,qualitativa,nominal
8,class,Classe do passageiro no navio,qualitativa,ordinal
9,who,"Indica se é homem, mulher ou criança",qualitativa,nominal


## 2. Preparação de dados

Nesta etapa iremos realizar a imputação de dados faltantes na variável `age` por meio do método *hot-deck*, imputação de dados nas variáveis `embarked` e `embark_town` pela moda dessas variáveis, eliminaremos a coluna `deck`, pois há muitos dados faltantes e essa variável não será importante para a predição, e realizaremos a codificação e normalização dos dados.

In [3]:
variavel_alvo = 'alive'
variavel_excluida = 'deck'
variaveis_nominais = (dicionario.query("subtipo == 'nominal' and variavel != @variavel_alvo").variavel.to_list())
variaveis_ordinais = (dicionario.query("subtipo == 'ordinal' and variavel != @variavel_excluida").variavel.to_list())
variaveis_discretas = (dicionario.query("subtipo == 'discreta'").variavel.to_list())
variaveis_continuas = (dicionario.query("subtipo == 'contínua'").variavel.to_list())

X = df.drop(columns=[variavel_alvo, variavel_excluida], axis=1) # entradas
y = df[variavel_alvo] # saída

In [4]:
# removendo a coluna 'deck'
df.drop(columns=['deck'], inplace=True)
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


In [5]:
'''
# outra forma de fazer os tratamentos
# tratamento de dados faltantes em age
k_imputer = KNNImputer(n_neighbors=10).fit(df[variaveis_quantitativas])
df[variaveis_quantitativas] = k_imputer.transform(df[variaveis_quantitativas])

# tratamento de dados faltantes nas variáveis qualitativas
s_imputer = SimpleImputer(strategy='most_frequent').fit(df[variaveis_qualitativas])
df[variaveis_qualitativas] = s_imputer.transform(df[variaveis_qualitativas])
print(df.isna().sum())
'''

"\n# outra forma de fazer os tratamentos\n# tratamento de dados faltantes em age\nk_imputer = KNNImputer(n_neighbors=10).fit(df[variaveis_quantitativas])\ndf[variaveis_quantitativas] = k_imputer.transform(df[variaveis_quantitativas])\n\n# tratamento de dados faltantes nas variáveis qualitativas\ns_imputer = SimpleImputer(strategy='most_frequent').fit(df[variaveis_qualitativas])\ndf[variaveis_qualitativas] = s_imputer.transform(df[variaveis_qualitativas])\nprint(df.isna().sum())\n"

In [6]:
preprocess_continuas = Pipeline([
    ('missing', KNNImputer(n_neighbors=10)), # tratamento dados faltantes
    # dados quantitativos não precisam ser transformados
    ('normalization', StandardScaler())  # normalização
])

preprocess_discretas = Pipeline([
    # não há dados faltantes nas variáveis discretas
    # dados quantitativos não precisam ser transformados
    ('normalization', StandardScaler()) # normalização
])

preprocess_nominais = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # transformação de dados
    ('normalization', StandardScaler())  # normalização
])

preprocess_ordinais = Pipeline([
    # não há dados faltantes nas variáveis ordinais
    ('encoding', OrdinalEncoder()), # codificação de variáveis
    ('normalization', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('continuas', preprocess_continuas, variaveis_continuas),
    ('discretas', preprocess_discretas, variaveis_discretas),
    ('nominais', preprocess_nominais, variaveis_nominais),
    ('ordinais', preprocess_ordinais, variaveis_ordinais)
])

## 3. Metodologia

Iremos analisar quatro modelos, que serão testados utilizando o método de validação cruzada K-fold, visto que o conjunto de dados não é muito grande e o K-fold é mais adequado para datasets reduzidos. Os modelos a serem analisados são:
- K-Nearest-Neighbors
- Decision Tree
- Logistic Regression
- Naive Bayes

Cada um dos modelos será testado com os hiper-paraâmetros:
- **Acurácia** (accuracy): proporção entre os dados que foram corretamente previstos (como positivos ou negativos) com o total de dados observados;
- **Precisão** (precision): proporção entre dados corretamente previstos como positivos e o total de observações positivas.
- **Recall**: proporção entre dados corretamente previstos como positivos com o total de observações.
- **F1-score**: média entre precision e recall, portanto levando em conta tanto falsos positivos quanto falsos negativos.

## 4. Configuração do experimento

In [7]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = 0.2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'] 

max_iter = 6000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),
    ('Decision Tree', DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Logistic Regression', LogisticRegression(max_iter=max_iter, solver = 'liblinear'), {'C' : np.logspace(-4, 4, 20)}),
    ('Naive Bayes', GaussianNB(), {})
]

In [8]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    
    scores['model_name'] = [model_name] * n_splits_comparative_analysis
    display(pd.DataFrame(scores).select_dtypes(include=[float, int]).agg(['mean', 'std']))
    results = pd.concat([results, pd.DataFrame(scores)], ignore_index=True)

running K-Nearest Neighbors...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,2.108205,0.059483,0.983240,0.982079,0.982498,0.982218
std,0.056326,0.007181,0.008734,0.009569,0.009809,0.009549


running Decision Tree...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,0.483319,0.040191,1.0,1.0,1.0,1.0
std,0.126404,0.013522,0.0,0.0,0.0,0.0


running Logistic Regression...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,1.425062,0.041177,1.0,1.0,1.0,1.0
std,0.025063,0.011086,0.0,0.0,0.0,0.0


running Naive Bayes...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro
mean,0.182757,0.050202,1.0,1.0,1.0,1.0
std,0.014097,0.012094,0.0,0.0,0.0,0.0


In [9]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 4.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 4.1 Resultados gerais

### 4.2. Persistência do modelo

In [10]:
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "K-Nearest Neighbors"][0] 


model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y) #Seleciona o approach

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

Hiper parâmetros do modelo: {'n_neighbors': 3, 'weights': 'distance'}


In [11]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco

['../models/model.joblib']

# 5. Resultados e discussões

Apesar de ter o maior `fit_time`, o modelo K-Nearest Neighbors obteve os melhores e mais confiáveis resultados nos testes de acurácia, precisão, *recall* e *f1-score*.